In [82]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

import chromedriver_autoinstaller
import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# 배우 수상 이력 수집

In [2]:
#네이버 창 실행
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
driver.get("https://www.naver.com")
time.sleep(2)

In [4]:
# 영화배우 이름 검색
name = '이정재'

In [5]:
# 네이버 검색창에 "검색어" 검색
selector = "#query"
element = driver.find_element(By.CSS_SELECTOR, selector)    # 검색창 찾기
element.clear()                # 현재 검색창에 무언가 들어있으면 삭제
element.send_keys(name)   # 검색창에 검색어 입력
element.submit()  # 검색 클릭
time.sleep(1)

In [6]:
# 수상 클릭
driver.find_element(By.LINK_TEXT, "수상").click()
time.sleep(1)

In [10]:
#수상 갯수 세기
#main_pack > section > div.cm_content_wrap._cm_content_area_prize_tab > div > div > div.middle_title > h3 > span
#class 안에 "내용있음" this_text_number

selector = ".this_text_number"
prize_cnt = int(driver.find_element(By.CSS_SELECTOR, selector).text)
prize_cnt

37

In [15]:
# 1page의 첫번째 수상내역 텍스트 확인   #\+엔터 코드 내림
selector = "#main_pack > section > div.cm_content_wrap._cm_content_area_prize_tab > div > div > div._panel_wrapper > \
div:nth-child(1) > dl > div:nth-child(1) > dd > ul > li > span"
driver.find_element(By.CSS_SELECTOR, selector).text

'제13회 아름다운예술인상 영화예술인상'

In [ ]:
#main_pack > section > div.cm_content_wrap._cm_content_area_prize_tab > div > div > div._panel_wrapper > div:nth-child(1) > dl > div:nth-child(2) > dd > ul > li > span

In [40]:
# 반복문으로 1page 수상내역 전체 텍스트 수집 후 리스트로 저장
text_list = []

for i in range(1, 9):  # 한 페이지당 최대 8개 
    selector = f"#main_pack > section > div.cm_content_wrap._cm_content_area_prize_tab > div > div > div._panel_wrapper > \
    div:nth-child(1) > dl > div:nth-child({i}) > dd > ul > li > span"
    prize_text = driver.find_element(By.CSS_SELECTOR, selector).text
    text_list.append(prize_text)
    # print(prize_text)
text_list

['제13회 아름다운예술인상 영화예술인상',
 '2022 하와이영화비평가협회상 신인감독상 (헌트)',
 '제19회 한국이미지상 디딤돌상',
 '금관문화훈장',
 '제43회 청룡영화상 신인감독상 (헌트)',
 '제42회 한국영화평론가협회상 신인감독상 (헌트)',
 '제31회 부일영화상 신인감독상 (헌트)',
 '제74회 프라임타임 에미상 드라마 시리즈부문 남우주연상 (오징어 게임)']

In [22]:
#년도
selector = "#main_pack > section > div.cm_content_wrap._cm_content_area_prize_tab > div > div > div._panel_wrapper > div:nth-child(1) > dl > div:nth-child(1) > dt"
driver.find_element(By.CSS_SELECTOR, selector).text

'2023'

In [41]:
# 반복문으로 1page 안의 전체 연도 수집 후 리스트로 저장
year_list = []

for i in range(1, 9):
    selector = f"#main_pack > section > div.cm_content_wrap._cm_content_area_prize_tab > div > div > div._panel_wrapper > \
    div:nth-child(1) > dl > div:nth-child({i}) > dt"
    year = driver.find_element(By.CSS_SELECTOR, selector).text
    year_list.append(year)
    # print(year)
year_list

['2023', '2023', '2023', '2022', '2022', '2022', '2022', '2022']

In [43]:
pd.DataFrame({'수상':text_list, '년도':year_list})

,수상,년도
0,제13회 아름다운예술인상 영화예술인상,2023
1,2022 하와이영화비평가협회상 신인감독상 (헌트),2023
2,제19회 한국이미지상 디딤돌상,2023
3,금관문화훈장,2022
4,제43회 청룡영화상 신인감독상 (헌트),2022
5,제42회 한국영화평론가협회상 신인감독상 (헌트),2022
6,제31회 부일영화상 신인감독상 (헌트),2022
7,제74회 프라임타임 에미상 드라마 시리즈부문 남우주연상 (오징어 게임),2022


# page 바꾸면서 수상내역 크롤링

In [51]:
✨#페이지 이동  ##class 앞에 점찍고 사이띄어쓰기에도 점  클래스 아이디 있으면 태그 작성 안해도된다
selector = ".pg_next.on"
driver.find_element(By.CSS_SELECTOR, selector).click()

In [ ]:
#1page 에서 데이터 크롤링 다음 페이지 이동 클릭
#2page 에서 데이터 크롤링 ~마지막 페이지까지

In [56]:
✨# 총 페이지 번호 확인
selector = "._total"
page_num = int(driver.find_element(By.CSS_SELECTOR, selector).text)
page_num

5

In [ ]:
✨# 이중 for문
# try, except 구문(예외 처리) 
# - try 밑에를 실행하고, 에러나면 except 밑에를 실행하세요.
# break : 현재 반복문을 빠져나가세요.
#for: try: except: break: 순으로 4개 같이쓰인다고 생각하면 편할듯

In [63]:
✨✨✨✨✨✨✨✨✨# 페이지를 이동하면서 각 페이지의 수상내역을 수집 후 리스트로 저장하는 코드 
# 예외처리, 이중for문, break 

text_list = []

for j in range(1, 6):   # j=1,2,3,4,5
    try:
        for i in range(1, 9):  # 한 페이지당 최대 8개 
            try:  # 다음을 실행해, 에러나면 except 밑에 것을 실행해.
                selector = f"#main_pack > section > div.cm_content_wrap._cm_content_area_prize_tab > div > div > div._panel_wrapper > \
                div:nth-child({j}) > dl > div:nth-child({i}) > dd > ul > li > span"
                prize_text = driver.find_element(By.CSS_SELECTOR, selector).text
                text_list.append(prize_text)
                # print(prize_text)
            except:
                break           
        # 다음 페이지로 이동 클릭
        selector = ".pg_next.on"
        driver.find_element(By.CSS_SELECTOR, selector).click()
        time.sleep(2)
    except:
        break

In [71]:
#✨print(len(text_list))
#✨text_list

37


['제13회 아름다운예술인상 영화예술인상',
 '2022 하와이영화비평가협회상 신인감독상 (헌트)',
 '제19회 한국이미지상 디딤돌상',
 '금관문화훈장',
 '제43회 청룡영화상 신인감독상 (헌트)',
 '제42회 한국영화평론가협회상 신인감독상 (헌트)',
 '제31회 부일영화상 신인감독상 (헌트)',
 '제74회 프라임타임 에미상 드라마 시리즈부문 남우주연상 (오징어 게임)',
 '제1회 청룡시리즈어워즈 드라마부문 남우주연상 (오징어 게임)',
 '제27회 크리틱스 초이스 시상식 TV드라마시리즈부문 남우주연상 (오징어 게임)',
 '제37회 인디펜던트 스피릿 어워드 TV부문 남우주연상 (오징어 게임)',
 '제28회 미국배우조합상 TV드라마시리즈부문 남우주연상 (오징어 게임)',
 '아시아 아티스트 어워즈 대상 올해의 배우',
 '아시아 아티스트 어워즈 배우부문 페뷸러스상',
 '아시아 아티스트 어워즈 배우부문 핫트렌드상',
 '아시아 아티스트 어워즈 대상 올해의 배우 영화부문',
 '스타일아이콘어워즈 본상',
 '제3회 마리끌레르 아시아 스타 어워즈 올해의 배우상',
 '제24회 부일영화상 남우주연상',
 '제51회 백상예술대상 InStyle상',
 '제50회 백상예술대상 영화부문 남자 조연상',
 '제33회 한국영화평론가협회상 CJ CGV 스타상',
 '제34회 청룡영화상 남우조연상',
 '제50회 대종상영화제 하나금융그룹 인기상',
 '제31회 판타스포르토 국제영화제 감독 주간 섹션 남우주연상',
 "제4회 Mnet 20's Choice 가장 영향력 있는 스타 20인",
 '제1회 스타일아이콘어워즈 본상',
 '제29회 황금촬영상 시상식 연기대상',
 '제38회 저축의 날 국무총리표창',
 '제35회 백상예술대상 TV부문 인기상',
 '제20회 청룡영화상 남우주연상',
 '제19회 한국영화평론가협회상 남자연기상',
 '제33회 백상예술대상 인기상',
 '제16회 청룡영화상 신인남우상',
 '제15회 한국영화평론가협회상 신인연기상',
 '제33회 대종

In [ ]:
# 페이지 이동하면서 연도 수집 후 리스트에 저장
# 데이터프레임, csv로 저장

In [90]:
year_list = []

for j in range(1, 6):   # j=1,2,3,4,5
    try:
        for i in range(1, 100):  # 한 페이지당 최대 8개 
            try:  # 다음을 실행해, 에러나면 except 밑에 것을 실행해.
                selector = f"#main_pack > section > div.cm_content_wrap._cm_content_area_prize_tab > div > div > div._panel_wrapper >\
                div:nth-child({j}) > dl > div:nth-child({i}) > dt"
                prize_year = int(driver.find_element(By.CSS_SELECTOR, selector).text)
                year_list.append(prize_year)
                # print(prize_text)
            except:
                break           
        # 다음 페이지로 이동 클릭
        selector = ".pg_next.on"
        driver.find_element(By.CSS_SELECTOR, selector).click()
        time.sleep(2)
    except:
        break

In [91]:
print(len(year_list))
year_list

37


[2023,
 2023,
 2023,
 2022,
 2022,
 2022,
 2022,
 2022,
 2022,
 2022,
 2022,
 2022,
 2021,
 2021,
 2021,
 2020,
 2016,
 2015,
 2015,
 2015,
 2014,
 2013,
 2013,
 2013,
 2011,
 2010,
 2008,
 2006,
 2001,
 1999,
 1999,
 1999,
 1997,
 1995,
 1995,
 1995,
 1995]

In [92]:
# 데이터프레임, csv로 저장  #변수 꼭 만들어야함

print(len(year_list))
df_py = pd.DataFrame({"수상":text_list , "년도":year_list })
df_py

37


,수상,년도
0,제13회 아름다운예술인상 영화예술인상,2023
1,2022 하와이영화비평가협회상 신인감독상 (헌트),2023
2,제19회 한국이미지상 디딤돌상,2023
3,금관문화훈장,2022
4,제43회 청룡영화상 신인감독상 (헌트),2022
5,제42회 한국영화평론가협회상 신인감독상 (헌트),2022
6,제31회 부일영화상 신인감독상 (헌트),2022
7,제74회 프라임타임 에미상 드라마 시리즈부문 남우주연상 (오징어 게임),2022
8,제1회 청룡시리즈어워즈 드라마부문 남우주연상 (오징어 게임),2022
9,제27회 크리틱스 초이스 시상식 TV드라마시리즈부문 남우주연상 (오징어 게임),2022


In [95]:
#csv로 저장
df_py.to_csv("aa.csv", encoding="cp949")
#데이터저장변수.to_csv("파일이름",encoding="cp949"  /utf-8-sig 

In [98]:
#min()은 "minimum" 의 약자로, 최솟값을 의미해요.
df_py['년도'].min()

1995

In [ ]:
# 이전 페이지로 돌아가기
driver.back()

In [ ]:
# [실습2] 프로필, 경력사항 크롤링